In [ ]:
!pip install pycoingecko requests tiktoken cohere openai pinecone-client langchain apify-client langchainhub pymupdf lark

In [ ]:
import os

os.environ["APIFY_API_TOKEN"]   = "APIFY_API_TOKEN"
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

os.environ["VECTARA_CUSTOMER_ID"] = "VECTARA_CUSTOMER_ID"
os.environ["VECTARA_CORPUS_ID"] = "VECTARA_CORPUS_ID"
os.environ["VECTARA_API_KEY"] = "VECTARA_API_KEY"

In [ ]:
import numpy as np
import datetime as dt
import openai
import re
import time
import pinecone

# Step 2 - Import Data

## Data Source 1/3: APIFY Google Search Result

In [ ]:
queries = "Semiconductor education in Korea"

In [ ]:
from apify_client import ApifyClient

# Initialize the ApifyClient with your API token
client = ApifyClient(os.getenv("APIFY_API_TOKEN"))

# Prepare the Actor input
run_input = {
    "queries": queries,
    "maxPagesPerQuery": 1,
    "resultsPerPage": 100,
    "customDataFunction": """async ({ input, $, request, response, html }) => {
  return {
    pageTitle: $('title').text(),
  };
};""",
}

# Run the Actor and wait for it to finish
run = client.actor("apify/google-search-scraper").call(run_input=run_input)

# Fetch and print Actor results from the run's dataset (if there are any)
loader = client.dataset(run["defaultDatasetId"]).iterate_items()

In [ ]:
temp = list()
results = list()

for i in loader:
  temp.append(i)

print(temp)

In [ ]:
data = temp[0]['organicResults']

In [ ]:
data

In [ ]:
# dates = [result.get('date') for result in temp[0]['organicResults'] if 'date' in result]

In [ ]:
# print(dates)

In [ ]:
from datetime import datetime

In [ ]:
#2023년 데이터만
filtered_data = [item for item in data if 'date' in item and datetime.fromisoformat(item['date'][:-1]).year == 2023]

In [ ]:
urls = []
dates = []
for item in filtered_data:
  urls.append(item['url'])
  dates.append(item['date'])

In [ ]:
urls

In [ ]:
dates

## Data Source 2/3:APIFY Web Contents Crawling

In [ ]:
from langchain.utilities import ApifyWrapper
from langchain.document_loaders.base import Document
import os

# Initialize the ApifyClient with your API token
apify = ApifyWrapper()

startUrls = [{"url": url} for url in urls]

#print(startUrls)

# Prepare the Actor input
run_input = {
    "startUrls": startUrls,
    "includeUrlGlobs": [],
    "excludeUrlGlobs": [],
    "initialCookies": [],
    "proxyConfiguration": { "useApifyProxy": True },
    "removeElementsCssSelector": """nav, footer, script, style, noscript, svg,
[role=\"alert\"],
[role=\"banner\"],
[role=\"dialog\"],
[role=\"alertdialog\"],
[role=\"region\"][aria-label*=\"skip\" i],
[aria-modal=\"true\"]""",
    "clickElementsCssSelector": "[aria-expanded=\"false\"]",
}

# Run the Actor and wait for it to finish
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input=run_input,
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"], "date": item["date"]}
    ),
)

## Data Source 3/3 - Vectara Initialize

In [ ]:
from langchain.vectorstores import Vectara

vectara = Vectara(
    vectara_customer_id = os.getenv("VECTARA_CUSTOMER_ID"),
    vectara_corpus_id   = os.getenv("VECTARA_CORPUS_ID"),
    vectara_api_key     = os.getenv("VECTARA_API_KEY")
)

In [ ]:
documents = loader.load()

In [ ]:
documents

In [ ]:
vectera = Vectara.from_documents(
    documents,
    embedding=None,
    doc_metadata={"category":"knowledgebase", "keyword":queries}
)